In [34]:
import os

print(os.getcwd())

/home/ai/playground


# 0. 전체 출력 결과 확인

In [35]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Fetch the API key from the environment variable
api_key = os.getenv('OPENAI_API_KEY')

# Define system prompt
system_prompt = {
    "role": "system",
    "content": '''
    <Basic>
        1) 사용자 질문(Question)에 따라 핵심 정보를 추출하기 위해 키워드, 즉 슬롯을 추출하기 위한 프롬프트를 작성하고 싶어.
        2) 다음과 같은 카테고리들을 "슬롯"이라고 명칭할 것이고 슬롯 목록은 다음과 같아: "개인화요소", "핵심 키워드", "감정/분위기", "시기/상황", "독서 목적", "대상 독자", "난이도", "분량"
    </Basic>
    <Check>
        사용자 질문에서 핵심 정보를 추출하기 위해 다음 단계를 따를 것:
        1) 질문의 핵심 주제나 의도를 파악하여 요청된 조건과 필요한 정보를 분리
        2) 의미를 명확하게 전달하는 중요 단어를 선택하여 각 슬롯에 적합한 키워드를 추출
            2-1. 질문에서 의미를 명확하게 전달하는 주요 명사, 형용사, 동사를 선택하여 적절한 슬롯에 매칭
            2-2. 불필요한 연결어, 조사(은, 는, 이, 가 등) 및 일반적인 표현(예: 좋은 책)은 배제
        3) 추출된 키워드는 각 슬롯 목록과 매칭되도록 구조화해서 정리(슬롯에 들어갈만한 쿼리값을 JSON형식으로 구조화하여 전달)
            3-1. "개인화요소"는 메타정보와 매칭되는 경우 'Y', 아닌 경우는 'N'으로 출력하며 매칭되는 경우 해당 정보를 리스트 형식으로 제공
            3-2. "핵심 키워드"
            3-3. "감정/분위기"
            3-4. "시기/상황"의 경우 오늘은 2025-02-10이며, 출간일 제한이 필요한 경우 yyyy-mm-dd 형태와 이전_이후 여부로 출력
            3-5. "독서 목적"
            3-6. "대상 독자"
            3-7. "난이도"의 경우 1부터 5단계로 정의
            3-8. "분량"의 경우 평균적인 일반단행본 페이지수를 고려하여 적절한 int값으로 제공(너무 짧은 분량이면 대상 도서가 없을 수도 있으니, 단편집 같은 적절한 포맷도 카테고리로 고려)

        4) 3-1부터 3-8까지 매칭되지 않는 필드는 일단 NA로 반환한 후 <Notes> 조건을 반영해줘
    </Check>
    <Notes>
        1) 매칭되지 않는 필드, 즉 "NA"로 반환되는 슬롯 값의 경우 값 또는 키값 모두 출력되지 않을 것
        2) 날짜 포맷과 난이도 레벨, 페이지 수는 반드시 명시된 기준에 맞출 것
        3) JSON은 참고할 부분을 키 값으로 해서 하나로 출력할 것
    </Notes>
    <Output>
        질문에 대한 슬롯 매핑의 예시는 다음과 같습니다:

        {
        "개인화요소": "N",
        "핵심 키워드": ["단시간 독서", "최근 출간", "추운 날씨"],
        "감정/분위기": ["따뜻한", "잔잔한"],
        "시기/상황": {
            "이전_이후": "이후",
            "기준일": "2024-11-07"
        },
        "독서 목적": ["힐링", "가벼운 독서"],
        "대상 독자": "NA",
        "난이도": 2,
        "분량": {
            "최소_페이지": 50,
            "최대_페이지": 120
        }
        }
    '''
}

# Function to create messages for the OpenAI API
def create_messages(user_messages):
    return [system_prompt] + [{"role": "user", "content": msg} for msg in user_messages]

# Test user input
user_messages = ['30분만에 읽을 만한 책 중 최근 3개월 이내 출판된 책으로 요즘 같은 추운 날씨에 읽기 좋은 책 추천해줘.']

# Define function to test with multiple models
def test_models(models):
    results = {}
    for model_name in models:
        # Set the model in the environment
        os.environ['LLM_MODEL'] = model_name
        
        # Create OpenAI client with current model
        client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
        
        # Create OpenAI messages
        current_messages = create_messages(user_messages)
        
        # Make API call
        response = client.chat.completions.create(
            model=model_name,
            messages=current_messages,
            stream=False
        )
        
        # Store the result
        results[model_name] = response.choices[0].message.content
    
    return results

# Models to test
models_to_test = ['gpt-4o-mini', 'gpt-4o', 'o3-mini']

# Test and print results
output_results = test_models(models_to_test)

# Display results for each model
for model, output in output_results.items():
    print(f"### Output for model {model} ###\n")
    print(output)
    print("\n" + "="*50 + "\n")


### Output for model gpt-4o-mini ###

{
    "개인화요소": "N",
    "핵심 키워드": ["30분 독서", "최근 3개월 출간", "추운 날씨"],
    "감정/분위기": ["따뜻한", "편안한"],
    "시기/상황": {
        "이전_이후": "이후",
        "기준일": "2025-02-10"
    },
    "독서 목적": ["힐링", "가벼운 독서"],
    "대상 독자": "NA",
    "난이도": 1,
    "분량": {
        "최소_페이지": 30,
        "최대_페이지": 50
    }
}


### Output for model gpt-4o ###

{
"개인화요소": "N",
"핵심 키워드": ["단시간 독서", "최근 출간", "추운 날씨"],
"감정/분위기": ["따뜻한", "잔잔한"],
"시기/상황": {
    "이전_이후": "이후",
    "기준일": "2024-11-10"
},
"독서 목적": ["힐링", "가벼운 독서"],
"난이도": 2,
"분량": {
    "최소_페이지": 50,
    "최대_페이지": 100
}
}


### Output for model o3-mini ###

{
  "핵심 키워드": ["30분", "최근 3개월 이내 출판", "추운 날씨"],
  "감정/분위기": ["따뜻한"],
  "시기/상황": {
    "이전_이후": "이후",
    "기준일": "2025-02-10"
  },
  "독서 목적": ["간편 독서"],
  "분량": {
    "최소_페이지": 50,
    "최대_페이지": 120
  }
}


